# Driver Insurance Prediction

Porto Seguro’s Safe Driver Prediction
Predict if a driver will file an insurance claim next year.

This challenge is a binary classification task. We are predicting the probability that the given driver will file an insurance claim or not.

I'll have to implement the normalized Gini coefficient measure for scoring.
Update: See Gini function for implementation, from external source

See [here](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction#evaluation) on Kaggle.

In [1]:
# Python 2 and 3 support
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# Constant seed
np.random.seed(42)

In [2]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

## Examining the data

TODO: Implement feature selection/augmentation and fix NA (data with -1), one-hot encoding for cat data

In [3]:
import pandas as pd

DATA_PATH='../data'

In [4]:
train = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))

In [9]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [11]:
train.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.115549e+06,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488027e+06,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
corr_matrix = train.corr()

In [3]:
# Uncomment to see
#corr_matrix["target"].sort_values(ascending=False)

In [1]:
# Uncomment to see
#train.info()

## Loading Data

Ps_car_13 seems to be the most important feature, but there a lot of good quality features here, so we'll keep them all for now.

Next we have to create a custom dataset class and a dataloader for PyTorch.

I've also created two new files, train_subset.csv and test_subset.csv because our test data doesn't actually come with labels. The first 500,000 examples go to the train subset and the other ~95,000 go to the test subset.

data_helper.py has the relevant code for generating the subsets, Jupyter can't handle all the output.

In [325]:
# For scikit-learn models
cols = [x for x in range(3,60)]
cols_final = [x for x in range(1,58)]
x_train = pd.read_csv(os.path.join(DATA_PATH, "train_subset.csv"), usecols=cols)
y_train = pd.read_csv(os.path.join(DATA_PATH, "train_subset.csv"), usecols=[2])

x_test = pd.read_csv(os.path.join(DATA_PATH, "test_subset.csv"), usecols=cols)
y_test = pd.read_csv(os.path.join(DATA_PATH, "test_subset.csv"), usecols=[2])

In [ ]:
x_train_all = pd.read_csv(os.path.join(DATA_PATH, "train.csv"), usecols=[x for x in range(2,59)])

In [ ]:
# For ANNs, minibatches of 100

y_test_mod_100 = pd.read_csv(os.path.join(DATA_PATH,"test_subset_mod_100.csv"), usecols=[2])

In [ ]:
# Final data does not have targets, must upload to Kaggle to see gini coeff.
x_final = pd.read_csv(os.path.join(DATA_PATH, "test.csv"), usecols=cols_final)
y_final_ids = pd.read_csv(os.path.join(DATA_PATH, "test.csv"), usecols=[0])

In [312]:
class DriverDataset(data.Dataset):
    def __init__(self, csv_path, cols=None):
        if cols == None:
            cols = [x for x in range(3,60)]
        
        self.x = pd.read_csv(csv_path, usecols=cols)
        self.y = pd.read_csv(csv_path, usecols=[2])
        
        self.x = self.x.as_matrix()
        self.y = self.y.as_matrix()
        
    def __getitem__(self, index):
        features = torch.from_numpy(self.x[index])
        label = torch.from_numpy(self.y[index])
        
        features, label = features.type(torch.FloatTensor), label.type(torch.FloatTensor)
        
        return features, label
            
    def __len__(self):
        return self.x.shape[0]

In [313]:
train_dataset = DriverDataset(os.path.join(DATA_PATH, "train_subset.csv"))

In [314]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)

In [315]:
test_dataset = DriverDataset(os.path.join(DATA_PATH, "test_subset_mod_100.csv"))

In [316]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=100, 
                                           shuffle=False,
                                           num_workers=2)

## Preprocessing

Run once, more will cause semantic errors

Ref: https://www.kaggle.com/yekenot/simple-stacker-lb-0-284

In [298]:
col_to_drop = x_train.columns[x_train.columns.str.startswith('ps_calc_')]
x_train_ex = x_train.copy()
x_test_ex = x_test.copy()

x_train_ex = x_train_ex.drop(col_to_drop, axis=1)  
x_test_ex = x_test_ex.drop(col_to_drop, axis=1)  

In [299]:
cat_features = [a for a in x_train_ex.columns if a.endswith('cat')]

In [300]:
for column in cat_features:
    temp = pd.get_dummies(pd.Series(x_train_ex[column]))
    x_train_ex = pd.concat([x_train_ex,temp],axis=1)
    x_train_ex = x_train_ex.drop([column],axis=1)
    
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(x_test_ex[column]))
    x_test_ex = pd.concat([x_test_ex,temp],axis=1)
    x_test_ex = x_test_ex.drop([column],axis=1)

In [301]:
lst = []

for x in range(len(x_train_ex.columns)):
    if type(x_train_ex.columns[x]) is not str:
        lst.append(x)
    else:
        lst.append(x_train_ex.columns[x])

In [302]:
x_train_ex.columns = lst

In [303]:
lst2 = []

for x in range(len(x_test_ex.columns)):
    if type(x_test_ex.columns[x]) is not str:
        lst2.append(x)
    else:
        lst2.append(x_test_ex.columns[x])

In [304]:
x_test_ex.columns = lst2

## Gini function

In [21]:
# calculate gini, ref: http://shichaoji.com/tag/train-model-predictive-insurance-auto-claim-gini-scikit-learn/
def gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(1/n_samples, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return G_pred/G_true

# Neural Network Models
Now we can try some models.

### PyTorch

In [317]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [461]:
input_size = 57
hidden_size = 1000 # 500
hidden_size_2 = 1000 # 200
num_classes = 1
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [462]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size_2)  
        self.relu2 = nn.ReLU()
        self.fc3= nn.Linear(hidden_size_2, num_classes)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [463]:
net = Net(input_size, hidden_size, num_classes)

In [464]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.2)

In [465]:
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        features = Variable(features)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/5000], Loss: 0.1810
Epoch [1/5], Step [200/5000], Loss: 0.1852
Epoch [1/5], Step [300/5000], Loss: 0.1378
Epoch [1/5], Step [400/5000], Loss: 0.1041
Epoch [1/5], Step [500/5000], Loss: 0.1008
Epoch [1/5], Step [600/5000], Loss: 0.2267
Epoch [1/5], Step [700/5000], Loss: 0.2003
Epoch [1/5], Step [800/5000], Loss: 0.2090
Epoch [1/5], Step [900/5000], Loss: 0.0657
Epoch [1/5], Step [1000/5000], Loss: 0.0753
Epoch [1/5], Step [1100/5000], Loss: 0.1283
Epoch [1/5], Step [1200/5000], Loss: 0.0638
Epoch [1/5], Step [1300/5000], Loss: 0.0938
Epoch [1/5], Step [1400/5000], Loss: 0.1340
Epoch [1/5], Step [1500/5000], Loss: 0.1071
Epoch [1/5], Step [1600/5000], Loss: 0.3329
Epoch [1/5], Step [1700/5000], Loss: 0.2111
Epoch [1/5], Step [1800/5000], Loss: 0.1778
Epoch [1/5], Step [1900/5000], Loss: 0.2634
Epoch [1/5], Step [2000/5000], Loss: 0.0962
Epoch [1/5], Step [2100/5000], Loss: 0.2068
Epoch [1/5], Step [2200/5000], Loss: 0.1375
Epoch [1/5], Step [2300/5000], Loss: 0.17

Epoch [4/5], Step [3900/5000], Loss: 0.1765
Epoch [4/5], Step [4000/5000], Loss: 0.0639
Epoch [4/5], Step [4100/5000], Loss: 0.2522
Epoch [4/5], Step [4200/5000], Loss: 0.2131
Epoch [4/5], Step [4300/5000], Loss: 0.1923
Epoch [4/5], Step [4400/5000], Loss: 0.1398
Epoch [4/5], Step [4500/5000], Loss: 0.1808
Epoch [4/5], Step [4600/5000], Loss: 0.2489
Epoch [4/5], Step [4700/5000], Loss: 0.1056
Epoch [4/5], Step [4800/5000], Loss: 0.1653
Epoch [4/5], Step [4900/5000], Loss: 0.1659
Epoch [4/5], Step [5000/5000], Loss: 0.2657
Epoch [5/5], Step [100/5000], Loss: 0.1892
Epoch [5/5], Step [200/5000], Loss: 0.1263
Epoch [5/5], Step [300/5000], Loss: 0.2556
Epoch [5/5], Step [400/5000], Loss: 0.2903
Epoch [5/5], Step [500/5000], Loss: 0.2729
Epoch [5/5], Step [600/5000], Loss: 0.2359
Epoch [5/5], Step [700/5000], Loss: 0.1316
Epoch [5/5], Step [800/5000], Loss: 0.1908
Epoch [5/5], Step [900/5000], Loss: 0.0624
Epoch [5/5], Step [1000/5000], Loss: 0.1376
Epoch [5/5], Step [1100/5000], Loss: 0.20

In [466]:
t = torch.from_numpy(x_test.as_matrix())
t = Variable(t.type(torch.FloatTensor))
y_pred_pytorch = net(t)

In [467]:
y_pred_pytorch = y_pred_pytorch.data.numpy()

In [468]:
gini(y_test.squeeze(), y_pred_pytorch.squeeze())

0.15914445828793219

# Scikit-learn Models

In [40]:
# TODO: Implement scaling?
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

### Linear Regression

In [120]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression(normalize=True)
lin_reg.fit(x_train,y_train.values.ravel())

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [121]:
y_pred_lin = lin_reg.predict(x_test)
accuracy_score(y_test, np.around(y_pred_lin))

0.9633134478847204

In [122]:
gini(y_test.squeeze(), y_pred_lin)

0.25627469557366445

### KNN

In [136]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor(n_jobs=-1, weights='distance', n_neighbors=3)
knn_reg.fit(x_train, y_train.values.ravel())

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
          weights='distance')

In [140]:
y_pred_knn = knn_reg.predict(x_test)

In [142]:
accuracy_score(y_test, np.around(y_pred_knn))

0.95962693778095198

In [143]:
gini(y_test.squeeze(), y_pred_knn)

0.033494451354015103

### Decision Tree

In [113]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=5,random_state=42,
                              min_samples_leaf=1,max_leaf_nodes=None,
                             min_samples_split=5,
                              min_weight_fraction_leaf=.1)

tree_reg.fit(x_train, y_train.values.ravel())

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.1, presort=False, random_state=42,
           splitter='best')

In [114]:
y_pred_tree_reg = tree_reg.predict(x_test)


In [124]:
accuracy_score(y_test, np.around(y_pred_tree_reg))

0.9633134478847204

In [125]:
gini(y_test.squeeze(), y_pred_tree_reg)

0.16492586158559264

### Random Forests

In [127]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42, n_jobs=-1)
forest_reg.fit(x_train, y_train.values.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [129]:
y_pred_rfg = forest_reg.predict(x_test)
accuracy_score(y_test, y_rfc_pred)

0.96328193925135486

In [131]:
gini(y_test.squeeze(), y_pred_rfg)

0.071237502831335231

### XGBoost

In [123]:
from xgboost import XGBRegressor, XGBClassifier

xgb_reg = XGBRegressor(n_estimators=50, learning_rate=0.01)
xgb_reg.fit(x_train, y_train.values.ravel())

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=50, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [130]:
y_pred_xgb = xgb_reg.predict(x_test)

In [131]:
accuracy_score(y_test, np.around(y_pred_xgb))

0.9633134478847204

In [132]:
gini(y_test.squeeze(), y_pred_xgb)

0.21458708505631321

In [470]:
n_estimators = 200

xgb_clf = XGBClassifier(n_estimators=n_estimators,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.8, 
                        colsample_bytree=.8,
                        gamma=1,
                        reg_alpha=0,
                        reg_lambda=1,
                        nthread=2)

In [471]:
xgb_clf.fit(x_train, y_train.values.ravel())

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=1, learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=200, nthread=2,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [472]:
y_pred_xgb_clf = xgb_clf.predict_proba(x_test)

In [477]:
gini(y_test.squeeze(), y_pred_xgb_clf[:,1])

0.26942940805579246

### XGBoost Training

In [138]:
# apply xgboost
# ref: http://shichaoji.com/tag/train-model-predictive-insurance-auto-claim-gini-scikit-learn/
import xgboost as xgb
d_train = xgb.DMatrix(x_train, y_train)
d_valid = xgb.DMatrix(x_test, y_test)
d_test = xgb.DMatrix(x_test)

def use_gini(a, b):
    y = b.get_label()
    return 'gini', gini(y, a)
    
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [139]:
xgb_params = {}
xgb_params['objective'] = 'binary:logistic'
xgb_params['eta'] = 0.02
xgb_params['silent'] = True
xgb_params['max_depth'] = 6
xgb_params['subsample'] = 0.9
xgb_params['colsample_bytree'] = 0.9

In [140]:
clf = xgb.train(xgb_params, d_train, 2000, watchlist, early_stopping_rounds=100, 
                verbose_eval=50, 
                feval=use_gini, 
                maximize=True)

[0]	train-gini:0.228071	valid-gini:0.213673
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[50]	train-gini:0.273333	valid-gini:0.242636
[100]	train-gini:0.284109	valid-gini:0.247184
[150]	train-gini:0.303867	valid-gini:0.254895
[200]	train-gini:0.323846	valid-gini:0.262313
[250]	train-gini:0.341542	valid-gini:0.267128
[300]	train-gini:0.355644	valid-gini:0.271085
[350]	train-gini:0.368506	valid-gini:0.273667
[400]	train-gini:0.379483	valid-gini:0.275027
[450]	train-gini:0.389474	valid-gini:0.276059
[500]	train-gini:0.398574	valid-gini:0.276275
[550]	train-gini:0.407576	valid-gini:0.276711
[600]	train-gini:0.41523	valid-gini:0.276674
[650]	train-gini:0.423521	valid-gini:0.27702
[700]	train-gini:0.431673	valid-gini:0.277167
[750]	train-gini:0.438955	valid-gini:0.277165
[800]	train-gini:0.445401	valid-gini:0.277551
[850]	train-gini:0.453191	valid-gini:0.277618
[900]	train-gini:0.460168	valid

In [142]:
clf.best_iteration,clf.best_ntree_limit,clf.best_score

(952, 953, 0.278329)

In [143]:
y_pred_xgb_trained = clf.predict(d_test)

In [144]:
gini(y_test.squeeze(), y_pred_xgb_trained)

0.27798485264432765

In [145]:
clf.save_model(os.path.join('../xgboost_models','xgb_2_extradata.bst'))

### Catboost

In [146]:
from catboost import CatBoostRegressor

In [283]:
catboost = CatBoostRegressor(iterations=1200, learning_rate=0.02, depth=7, loss_function='RMSE', eval_metric='AUC', random_seed=42, od_type='Iter', od_wait=100) 
catboost = catboost.fit(x_train, y_train.values.ravel(), eval_set=(x_test, y_test.squeeze()), use_best_model=True, verbose=True)

Borders for float features generated
0:	learn 0.5878486121	test 0.5838554809	bestTest 0.5838554809		total: 541ms	remaining: 3h 18m 11s
1:	learn 0.6009435128	test 0.5980131713	bestTest 0.5980131713		total: 998ms	remaining: 3h 2m 59s
2:	learn 0.6029504679	test 0.5986196381	bestTest 0.5986196381		total: 1.45s	remaining: 2h 56m 48s
3:	learn 0.6094596616	test 0.6066359121	bestTest 0.6066359121		total: 1.89s	remaining: 2h 53m 17s
4:	learn 0.613784242	test 0.6096744517	bestTest 0.6096744517		total: 2.36s	remaining: 2h 52m 52s
5:	learn 0.6153262478	test 0.6115024376	bestTest 0.6115024376		total: 2.83s	remaining: 2h 53m 1s
6:	learn 0.6163856413	test 0.6120442613	bestTest 0.6120442613		total: 3.31s	remaining: 2h 53m 15s
7:	learn 0.617251341	test 0.6125133967	bestTest 0.6125133967		total: 3.78s	remaining: 2h 53m 20s
8:	learn 0.6171274057	test 0.6125564978	bestTest 0.6125564978		total: 4.25s	remaining: 2h 53m 6s
9:	learn 0.6169200889	test 0.6128940637	bestTest 0.6128940637		total: 4.75s	remaining:

83:	learn 0.6305928545	test 0.6249808884	bestTest 0.6249808884		total: 40.1s	remaining: 2h 54m 34s
84:	learn 0.6306254224	test 0.6250651023	bestTest 0.6250651023		total: 40.7s	remaining: 2h 54m 48s
85:	learn 0.6308882879	test 0.6252196871	bestTest 0.6252196871		total: 41.2s	remaining: 2h 54m 48s
86:	learn 0.6310265699	test 0.6252247561	bestTest 0.6252247561		total: 41.6s	remaining: 2h 54m 44s
87:	learn 0.6311990422	test 0.6253673299	bestTest 0.6253673299		total: 42.1s	remaining: 2h 54m 48s
88:	learn 0.6313275033	test 0.6254330093	bestTest 0.6254330093		total: 42.6s	remaining: 2h 54m 47s
89:	learn 0.631314236	test 0.6254332278	bestTest 0.6254332278		total: 43.1s	remaining: 2h 54m 46s
90:	learn 0.6313941436	test 0.6254132293	bestTest 0.6254332278		total: 43.6s	remaining: 2h 54m 50s
91:	learn 0.6314821665	test 0.6254929766	bestTest 0.6254929766		total: 44.1s	remaining: 2h 54m 51s
92:	learn 0.631584387	test 0.6254864374	bestTest 0.6254929766		total: 44.5s	remaining: 2h 54m 47s
93:	learn 0.

166:	learn 0.6379821698	test 0.6287359848	bestTest 0.6287359848		total: 1m 19s	remaining: 2h 53m 59s
167:	learn 0.6380559208	test 0.6287780137	bestTest 0.6287780137		total: 1m 20s	remaining: 2h 54m 1s
168:	learn 0.6381665437	test 0.6288505538	bestTest 0.6288505538		total: 1m 20s	remaining: 2h 53m 59s
169:	learn 0.638183301	test 0.6288569089	bestTest 0.6288569089		total: 1m 21s	remaining: 2h 53m 52s
170:	learn 0.6382143355	test 0.6288483938	bestTest 0.6288569089		total: 1m 21s	remaining: 2h 53m 52s
171:	learn 0.6382592981	test 0.6288597181	bestTest 0.6288597181		total: 1m 22s	remaining: 2h 53m 51s
172:	learn 0.6383156349	test 0.6288929604	bestTest 0.6288929604		total: 1m 22s	remaining: 2h 53m 52s
173:	learn 0.6383855847	test 0.6289524283	bestTest 0.6289524283		total: 1m 23s	remaining: 2h 53m 55s
174:	learn 0.6383887091	test 0.6289543636	bestTest 0.6289543636		total: 1m 23s	remaining: 2h 53m 45s
175:	learn 0.6384802148	test 0.6289969512	bestTest 0.6289969512		total: 1m 24s	remaining: 2h 

248:	learn 0.643796164	test 0.6311732233	bestTest 0.6311732233		total: 1m 59s	remaining: 2h 53m 47s
249:	learn 0.6438409359	test 0.6312043587	bestTest 0.6312043587		total: 1m 59s	remaining: 2h 53m 47s
250:	learn 0.6438344335	test 0.6312012062	bestTest 0.6312043587		total: 2m	remaining: 2h 53m 33s
251:	learn 0.6439100385	test 0.6312089783	bestTest 0.6312089783		total: 2m	remaining: 2h 53m 31s
252:	learn 0.6439846061	test 0.6312310993	bestTest 0.6312310993		total: 2m 1s	remaining: 2h 53m 31s
253:	learn 0.6440185366	test 0.6312331781	bestTest 0.6312331781		total: 2m 1s	remaining: 2h 53m 29s
254:	learn 0.6440799958	test 0.6312634302	bestTest 0.6312634302		total: 2m 2s	remaining: 2h 53m 28s
255:	learn 0.6441352822	test 0.6313109184	bestTest 0.6313109184		total: 2m 2s	remaining: 2h 53m 27s
256:	learn 0.6442172692	test 0.6313762607	bestTest 0.6313762607		total: 2m 2s	remaining: 2h 53m 25s
257:	learn 0.6443055119	test 0.6314172534	bestTest 0.6314172534		total: 2m 3s	remaining: 2h 53m 22s
258:	

330:	learn 0.6486982997	test 0.6326390049	bestTest 0.6326390049		total: 2m 39s	remaining: 2h 53m 32s
331:	learn 0.6487556608	test 0.63265552	bestTest 0.63265552		total: 2m 39s	remaining: 2h 53m 33s
332:	learn 0.6488113481	test 0.6326647092	bestTest 0.6326647092		total: 2m 40s	remaining: 2h 53m 34s
333:	learn 0.6488619016	test 0.6326859874	bestTest 0.6326859874		total: 2m 40s	remaining: 2h 53m 34s
334:	learn 0.6488709052	test 0.6327028989	bestTest 0.6327028989		total: 2m 41s	remaining: 2h 53m 35s
335:	learn 0.6489257655	test 0.6327322115	bestTest 0.6327322115		total: 2m 41s	remaining: 2h 53m 36s
336:	learn 0.6489882367	test 0.6327292431	bestTest 0.6327322115		total: 2m 42s	remaining: 2h 53m 35s
337:	learn 0.6489937406	test 0.6327311377	bestTest 0.6327322115		total: 2m 42s	remaining: 2h 53m 37s
338:	learn 0.6491456909	test 0.6327226477	bestTest 0.6327322115		total: 2m 43s	remaining: 2h 53m 36s
339:	learn 0.6491613289	test 0.6327245298	bestTest 0.6327322115		total: 2m 43s	remaining: 2h 53

...

1393:	learn 0.6897260532	test 0.6370849023	bestTest 0.6371262383		total: 11m 21s	remaining: 2h 47m 53s
1394:	learn 0.6897694106	test 0.6370772893	bestTest 0.6371262383		total: 11m 21s	remaining: 2h 47m 52s
1395:	learn 0.6897993567	test 0.6370996694	bestTest 0.6371262383		total: 11m 22s	remaining: 2h 47m 52s
1396:	learn 0.6898197646	test 0.6371026783	bestTest 0.6371262383		total: 11m 22s	remaining: 2h 47m 52s
1397:	learn 0.689848059	test 0.6370774891	bestTest 0.6371262383		total: 11m 23s	remaining: 2h 47m 51s
1398:	learn 0.6898974234	test 0.63704518	bestTest 0.6371262383		total: 11m 23s	remaining: 2h 47m 51s
1399:	learn 0.6899276891	test 0.6370430981	bestTest 0.6371262383		total: 11m 24s	remaining: 2h 47m 50s
1400:	learn 0.6899898568	test 0.6370536638	bestTest 0.6371262383		total: 11m 24s	remaining: 2h 47m 50s
1401:	learn 0.6900226868	test 0.6370825144	bestTest 0.6371262383		total: 11m 25s	remaining: 2h 47m 49s
1402:	learn 0.690048569	test 0.6370779448	bestTest 0.6371262383		total: 11m 

1473:	learn 0.692151453	test 0.6370461976	bestTest 0.6371634978		total: 12m	remaining: 2h 47m 18s
1474:	learn 0.6921874536	test 0.6370638301	bestTest 0.6371634978		total: 12m 1s	remaining: 2h 47m 17s
1475:	learn 0.6922556127	test 0.6370744707	bestTest 0.6371634978		total: 12m 1s	remaining: 2h 47m 17s
1476:	learn 0.6923217049	test 0.6370492065	bestTest 0.6371634978		total: 12m 2s	remaining: 2h 47m 16s
1477:	learn 0.69237188	test 0.6370534765	bestTest 0.6371634978		total: 12m 2s	remaining: 2h 47m 15s
1478:	learn 0.6923798519	test 0.6370550716	bestTest 0.6371634978		total: 12m 3s	remaining: 2h 47m 15s
1479:	learn 0.6924191044	test 0.6370419557	bestTest 0.6371634978		total: 12m 3s	remaining: 2h 47m 14s
1480:	learn 0.6924717528	test 0.6370475959	bestTest 0.6371634978		total: 12m 4s	remaining: 2h 47m 13s
1481:	learn 0.6924855151	test 0.6370452143	bestTest 0.6371634978		total: 12m 4s	remaining: 2h 47m 13s
1482:	learn 0.692487065	test 0.63704199	bestTest 0.6371634978		total: 12m 5s	remaining: 

1554:	learn 0.6946132435	test 0.6371670437	bestTest 0.6371715634		total: 12m 40s	remaining: 2h 46m 39s
1555:	learn 0.6946532678	test 0.6371905193	bestTest 0.6371905193		total: 12m 41s	remaining: 2h 46m 39s
1556:	learn 0.6946868909	test 0.6371830374	bestTest 0.6371905193		total: 12m 41s	remaining: 2h 46m 39s
1557:	learn 0.6947009017	test 0.6371966122	bestTest 0.6371966122		total: 12m 42s	remaining: 2h 46m 38s
1558:	learn 0.6947272084	test 0.6371810679	bestTest 0.6371966122		total: 12m 42s	remaining: 2h 46m 37s
1559:	learn 0.694749561	test 0.6371809867	bestTest 0.6371966122		total: 12m 42s	remaining: 2h 46m 37s
1560:	learn 0.6947870836	test 0.6371801002	bestTest 0.6371966122		total: 12m 43s	remaining: 2h 46m 36s
1561:	learn 0.6948237723	test 0.6371621119	bestTest 0.6371966122		total: 12m 43s	remaining: 2h 46m 36s
1562:	learn 0.6948645766	test 0.6371825411	bestTest 0.6371966122		total: 12m 44s	remaining: 2h 46m 35s
1563:	learn 0.6949231049	test 0.6371970429	bestTest 0.6371970429		total: 1

1635:	learn 0.6969018105	test 0.6372064007	bestTest 0.637218446		total: 13m 20s	remaining: 2h 46m 2s
1636:	learn 0.6969153402	test 0.6372031576	bestTest 0.637218446		total: 13m 20s	remaining: 2h 46m 2s
1637:	learn 0.6969322448	test 0.6372154245	bestTest 0.637218446		total: 13m 21s	remaining: 2h 46m 2s
1638:	learn 0.696933224	test 0.6372102837	bestTest 0.637218446		total: 13m 21s	remaining: 2h 46m 1s
1639:	learn 0.6969515505	test 0.637222098	bestTest 0.637222098		total: 13m 22s	remaining: 2h 46m 1s
1640:	learn 0.6969756521	test 0.637208002	bestTest 0.637222098		total: 13m 22s	remaining: 2h 46m
1641:	learn 0.6969814721	test 0.6372035634	bestTest 0.637222098		total: 13m 23s	remaining: 2h 46m
1642:	learn 0.6969934406	test 0.6371832278	bestTest 0.637222098		total: 13m 23s	remaining: 2h 45m 59s
1643:	learn 0.6970101791	test 0.6371849134	bestTest 0.637222098		total: 13m 24s	remaining: 2h 45m 59s
1644:	learn 0.6970336147	test 0.6371753027	bestTest 0.637222098		total: 13m 24s	remaining: 2h 45m 

1716:	learn 0.6991446286	test 0.6371966371	bestTest 0.6373141371		total: 14m	remaining: 2h 45m 26s
1717:	learn 0.699162533	test 0.6372222322	bestTest 0.6373141371		total: 14m	remaining: 2h 45m 26s
1718:	learn 0.6991781727	test 0.6372088479	bestTest 0.6373141371		total: 14m 1s	remaining: 2h 45m 25s
1719:	learn 0.6992112216	test 0.6372024491	bestTest 0.6373141371		total: 14m 1s	remaining: 2h 45m 25s
1720:	learn 0.6992305326	test 0.6372054269	bestTest 0.6373141371		total: 14m 2s	remaining: 2h 45m 24s
1721:	learn 0.6992673454	test 0.6371916492	bestTest 0.6373141371		total: 14m 2s	remaining: 2h 45m 23s
1722:	learn 0.6992894517	test 0.6372097406	bestTest 0.6373141371		total: 14m 3s	remaining: 2h 45m 23s
1723:	learn 0.6993051242	test 0.6372191171	bestTest 0.6373141371		total: 14m 3s	remaining: 2h 45m 23s
1724:	learn 0.699364809	test 0.6371936937	bestTest 0.6373141371		total: 14m 4s	remaining: 2h 45m 22s
1725:	learn 0.6994023888	test 0.6371763983	bestTest 0.6373141371		total: 14m 4s	remaining:

In [148]:
y_pred_catboost_trained = catboost.predict(x_test)

In [149]:
gini(y_test.squeeze(), y_pred_catboost_trained)

0.27859078732388493

In [150]:
catboost.save_model('../catboost_models/cat_1_extradata.bst', format="cbm")

### LightGBM

In [310]:
import lightgbm as lgb

lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 2000
lgb_params['max_depth'] = 4
lgb_params['seed'] = 42
lgb_params['metric'] = 'auc'

lgb_params2 = {}
lgb_params2['learning_rate'] = 0.02
lgb_params2['n_estimators'] = 650
lgb_params2['max_bin'] = 10
lgb_params2['subsample'] = 0.8
lgb_params2['subsample_freq'] = 10
lgb_params2['colsample_bytree'] = 0.8   
lgb_params2['min_child_samples'] = 500
lgb_params2['metric'] = 'auc'
lgb_params2['seed'] = 42

train_data = lgb.Dataset(x_train, label=y_train.values.ravel())
test_data = lgb.Dataset(x_test, label=y_test.squeeze())

In [311]:
lgb_clf = lgb.train(lgb_params, train_data, 2500, valid_sets=test_data, early_stopping_rounds=200)

[1]	valid_0's auc: 0.592951
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.59384
[3]	valid_0's auc: 0.594437
[4]	valid_0's auc: 0.600121
[5]	valid_0's auc: 0.601734
[6]	valid_0's auc: 0.603856
[7]	valid_0's auc: 0.603921
[8]	valid_0's auc: 0.605929
[9]	valid_0's auc: 0.606596
[10]	valid_0's auc: 0.606753
[11]	valid_0's auc: 0.607414
[12]	valid_0's auc: 0.607546
[13]	valid_0's auc: 0.608272
[14]	valid_0's auc: 0.610158
[15]	valid_0's auc: 0.610771
[16]	valid_0's auc: 0.610776
[17]	valid_0's auc: 0.611506
[18]	valid_0's auc: 0.612016
[19]	valid_0's auc: 0.612138
[20]	valid_0's auc: 0.611976
[21]	valid_0's auc: 0.612566
[22]	valid_0's auc: 0.612512
[23]	valid_0's auc: 0.613104
[24]	valid_0's auc: 0.613376
[25]	valid_0's auc: 0.613291
[26]	valid_0's auc: 0.613724
[27]	valid_0's auc: 0.613614
[28]	valid_0's auc: 0.61375
[29]	valid_0's auc: 0.614034
[30]	valid_0's auc: 0.614317
[31]	valid_0's auc: 0.614752
[32]	valid_0's auc: 0.615173
[33]	valid_0's auc: 

[281]	valid_0's auc: 0.63295
[282]	valid_0's auc: 0.633074
[283]	valid_0's auc: 0.633069
[284]	valid_0's auc: 0.633096
[285]	valid_0's auc: 0.633111
[286]	valid_0's auc: 0.633145
[287]	valid_0's auc: 0.633131
[288]	valid_0's auc: 0.633126
[289]	valid_0's auc: 0.633176
[290]	valid_0's auc: 0.633209
[291]	valid_0's auc: 0.633255
[292]	valid_0's auc: 0.633284
[293]	valid_0's auc: 0.633277
[294]	valid_0's auc: 0.633299
[295]	valid_0's auc: 0.633308
[296]	valid_0's auc: 0.633368
[297]	valid_0's auc: 0.633406
[298]	valid_0's auc: 0.633443
[299]	valid_0's auc: 0.633448
[300]	valid_0's auc: 0.633443
[301]	valid_0's auc: 0.633452
[302]	valid_0's auc: 0.633449
[303]	valid_0's auc: 0.633479
[304]	valid_0's auc: 0.633526
[305]	valid_0's auc: 0.633524
[306]	valid_0's auc: 0.63354
[307]	valid_0's auc: 0.633611
[308]	valid_0's auc: 0.633621
[309]	valid_0's auc: 0.633637
[310]	valid_0's auc: 0.633655
[311]	valid_0's auc: 0.633699
[312]	valid_0's auc: 0.633704
[313]	valid_0's auc: 0.633726
[314]	valid_

[559]	valid_0's auc: 0.63567
[560]	valid_0's auc: 0.635667
[561]	valid_0's auc: 0.635669
[562]	valid_0's auc: 0.635676
[563]	valid_0's auc: 0.635677
[564]	valid_0's auc: 0.635683
[565]	valid_0's auc: 0.635691
[566]	valid_0's auc: 0.635694
[567]	valid_0's auc: 0.635697
[568]	valid_0's auc: 0.635707
[569]	valid_0's auc: 0.635713
[570]	valid_0's auc: 0.635703
[571]	valid_0's auc: 0.635694
[572]	valid_0's auc: 0.635709
[573]	valid_0's auc: 0.635697
[574]	valid_0's auc: 0.635698
[575]	valid_0's auc: 0.635711
[576]	valid_0's auc: 0.635712
[577]	valid_0's auc: 0.635723
[578]	valid_0's auc: 0.635738
[579]	valid_0's auc: 0.63574
[580]	valid_0's auc: 0.63573
[581]	valid_0's auc: 0.635717
[582]	valid_0's auc: 0.635696
[583]	valid_0's auc: 0.635723
[584]	valid_0's auc: 0.635742
[585]	valid_0's auc: 0.63575
[586]	valid_0's auc: 0.63575
[587]	valid_0's auc: 0.635754
[588]	valid_0's auc: 0.635774
[589]	valid_0's auc: 0.635797
[590]	valid_0's auc: 0.635801
[591]	valid_0's auc: 0.635805
[592]	valid_0's

[838]	valid_0's auc: 0.63647
[839]	valid_0's auc: 0.636486
[840]	valid_0's auc: 0.636462
[841]	valid_0's auc: 0.636469
[842]	valid_0's auc: 0.636468
[843]	valid_0's auc: 0.63647
[844]	valid_0's auc: 0.636484
[845]	valid_0's auc: 0.636488
[846]	valid_0's auc: 0.636487
[847]	valid_0's auc: 0.6365
[848]	valid_0's auc: 0.636498
[849]	valid_0's auc: 0.636503
[850]	valid_0's auc: 0.636505
[851]	valid_0's auc: 0.636494
[852]	valid_0's auc: 0.636489
[853]	valid_0's auc: 0.636493
[854]	valid_0's auc: 0.636491
[855]	valid_0's auc: 0.636496
[856]	valid_0's auc: 0.63651
[857]	valid_0's auc: 0.636507
[858]	valid_0's auc: 0.636514
[859]	valid_0's auc: 0.636502
[860]	valid_0's auc: 0.636499
[861]	valid_0's auc: 0.636509
[862]	valid_0's auc: 0.636515
[863]	valid_0's auc: 0.636535
[864]	valid_0's auc: 0.636514
[865]	valid_0's auc: 0.636516
[866]	valid_0's auc: 0.636502
[867]	valid_0's auc: 0.636506
[868]	valid_0's auc: 0.636493
[869]	valid_0's auc: 0.63649
[870]	valid_0's auc: 0.63647
[871]	valid_0's a

[1113]	valid_0's auc: 0.636827
[1114]	valid_0's auc: 0.636821
[1115]	valid_0's auc: 0.636807
[1116]	valid_0's auc: 0.636807
[1117]	valid_0's auc: 0.636808
[1118]	valid_0's auc: 0.636817
[1119]	valid_0's auc: 0.636801
[1120]	valid_0's auc: 0.636808
[1121]	valid_0's auc: 0.636807
[1122]	valid_0's auc: 0.636799
[1123]	valid_0's auc: 0.636795
[1124]	valid_0's auc: 0.636783
[1125]	valid_0's auc: 0.63679
[1126]	valid_0's auc: 0.636781
[1127]	valid_0's auc: 0.636783
[1128]	valid_0's auc: 0.636788
[1129]	valid_0's auc: 0.636791
[1130]	valid_0's auc: 0.636784
[1131]	valid_0's auc: 0.636787
[1132]	valid_0's auc: 0.636765
[1133]	valid_0's auc: 0.636736
[1134]	valid_0's auc: 0.636726
[1135]	valid_0's auc: 0.636723
[1136]	valid_0's auc: 0.636714
[1137]	valid_0's auc: 0.636714
[1138]	valid_0's auc: 0.636716
[1139]	valid_0's auc: 0.636725
[1140]	valid_0's auc: 0.636733
[1141]	valid_0's auc: 0.636723
[1142]	valid_0's auc: 0.63672
[1143]	valid_0's auc: 0.636722
[1144]	valid_0's auc: 0.636729
[1145]	val

[1385]	valid_0's auc: 0.637056
[1386]	valid_0's auc: 0.637069
[1387]	valid_0's auc: 0.637066
[1388]	valid_0's auc: 0.637073
[1389]	valid_0's auc: 0.637079
[1390]	valid_0's auc: 0.637082
[1391]	valid_0's auc: 0.637072
[1392]	valid_0's auc: 0.637099
[1393]	valid_0's auc: 0.637093
[1394]	valid_0's auc: 0.637082
[1395]	valid_0's auc: 0.637071
[1396]	valid_0's auc: 0.637068
[1397]	valid_0's auc: 0.637076
[1398]	valid_0's auc: 0.637075
[1399]	valid_0's auc: 0.637073
[1400]	valid_0's auc: 0.637066
[1401]	valid_0's auc: 0.637075
[1402]	valid_0's auc: 0.637086
[1403]	valid_0's auc: 0.637084
[1404]	valid_0's auc: 0.637099
[1405]	valid_0's auc: 0.637101
[1406]	valid_0's auc: 0.63711
[1407]	valid_0's auc: 0.637098
[1408]	valid_0's auc: 0.637101
[1409]	valid_0's auc: 0.637121
[1410]	valid_0's auc: 0.637121
[1411]	valid_0's auc: 0.637116
[1412]	valid_0's auc: 0.637106
[1413]	valid_0's auc: 0.637113
[1414]	valid_0's auc: 0.637124
[1415]	valid_0's auc: 0.637116
[1416]	valid_0's auc: 0.637112
[1417]	va

[1656]	valid_0's auc: 0.637004
[1657]	valid_0's auc: 0.637025
[1658]	valid_0's auc: 0.637024
[1659]	valid_0's auc: 0.637003
[1660]	valid_0's auc: 0.636991
[1661]	valid_0's auc: 0.63699
[1662]	valid_0's auc: 0.636983
[1663]	valid_0's auc: 0.636987
[1664]	valid_0's auc: 0.637005
[1665]	valid_0's auc: 0.637005
[1666]	valid_0's auc: 0.637016
[1667]	valid_0's auc: 0.637021
[1668]	valid_0's auc: 0.637008
[1669]	valid_0's auc: 0.637001
[1670]	valid_0's auc: 0.637003
[1671]	valid_0's auc: 0.637
[1672]	valid_0's auc: 0.636998
[1673]	valid_0's auc: 0.636993
[1674]	valid_0's auc: 0.636993
[1675]	valid_0's auc: 0.637001
[1676]	valid_0's auc: 0.637017
[1677]	valid_0's auc: 0.637072
[1678]	valid_0's auc: 0.637059
[1679]	valid_0's auc: 0.637057
[1680]	valid_0's auc: 0.637066
[1681]	valid_0's auc: 0.637066
[1682]	valid_0's auc: 0.637078
[1683]	valid_0's auc: 0.637077
[1684]	valid_0's auc: 0.637067
[1685]	valid_0's auc: 0.637073
[1686]	valid_0's auc: 0.637068
[1687]	valid_0's auc: 0.637071
[1688]	valid

In [307]:
lgb_clf2 = lgb.train(lgb_params2, train_data, 2500, valid_sets=test_data, early_stopping_rounds=200)

[1]	valid_0's auc: 0.606865
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.610625
[3]	valid_0's auc: 0.614963
[4]	valid_0's auc: 0.614511
[5]	valid_0's auc: 0.617115
[6]	valid_0's auc: 0.617504
[7]	valid_0's auc: 0.62002
[8]	valid_0's auc: 0.61962
[9]	valid_0's auc: 0.619623
[10]	valid_0's auc: 0.620615
[11]	valid_0's auc: 0.621185
[12]	valid_0's auc: 0.62143
[13]	valid_0's auc: 0.621668
[14]	valid_0's auc: 0.622225
[15]	valid_0's auc: 0.622185
[16]	valid_0's auc: 0.622025
[17]	valid_0's auc: 0.621881
[18]	valid_0's auc: 0.622391
[19]	valid_0's auc: 0.62225
[20]	valid_0's auc: 0.622713
[21]	valid_0's auc: 0.622839
[22]	valid_0's auc: 0.6232
[23]	valid_0's auc: 0.623297
[24]	valid_0's auc: 0.623368
[25]	valid_0's auc: 0.623337
[26]	valid_0's auc: 0.623594
[27]	valid_0's auc: 0.623825
[28]	valid_0's auc: 0.624179
[29]	valid_0's auc: 0.624657
[30]	valid_0's auc: 0.624967
[31]	valid_0's auc: 0.624949
[32]	valid_0's auc: 0.625131
[33]	valid_0's auc: 0.62

[278]	valid_0's auc: 0.635922
[279]	valid_0's auc: 0.635949
[280]	valid_0's auc: 0.635978
[281]	valid_0's auc: 0.635984
[282]	valid_0's auc: 0.635956
[283]	valid_0's auc: 0.635943
[284]	valid_0's auc: 0.635982
[285]	valid_0's auc: 0.635976
[286]	valid_0's auc: 0.636026
[287]	valid_0's auc: 0.635994
[288]	valid_0's auc: 0.636045
[289]	valid_0's auc: 0.636079
[290]	valid_0's auc: 0.63608
[291]	valid_0's auc: 0.636059
[292]	valid_0's auc: 0.636091
[293]	valid_0's auc: 0.63612
[294]	valid_0's auc: 0.636125
[295]	valid_0's auc: 0.636119
[296]	valid_0's auc: 0.636113
[297]	valid_0's auc: 0.63612
[298]	valid_0's auc: 0.636108
[299]	valid_0's auc: 0.636106
[300]	valid_0's auc: 0.636113
[301]	valid_0's auc: 0.636112
[302]	valid_0's auc: 0.636163
[303]	valid_0's auc: 0.636199
[304]	valid_0's auc: 0.636237
[305]	valid_0's auc: 0.636218
[306]	valid_0's auc: 0.636248
[307]	valid_0's auc: 0.636298
[308]	valid_0's auc: 0.636308
[309]	valid_0's auc: 0.636299
[310]	valid_0's auc: 0.636347
[311]	valid_0

[557]	valid_0's auc: 0.637089
[558]	valid_0's auc: 0.637066
[559]	valid_0's auc: 0.637024
[560]	valid_0's auc: 0.637001
[561]	valid_0's auc: 0.636987
[562]	valid_0's auc: 0.636997
[563]	valid_0's auc: 0.637022
[564]	valid_0's auc: 0.637072
[565]	valid_0's auc: 0.637084
[566]	valid_0's auc: 0.637098
[567]	valid_0's auc: 0.637075
[568]	valid_0's auc: 0.637088
[569]	valid_0's auc: 0.637087
[570]	valid_0's auc: 0.637116
[571]	valid_0's auc: 0.637108
[572]	valid_0's auc: 0.637131
[573]	valid_0's auc: 0.637107
[574]	valid_0's auc: 0.637074
[575]	valid_0's auc: 0.637088
[576]	valid_0's auc: 0.637097
[577]	valid_0's auc: 0.637119
[578]	valid_0's auc: 0.637118
[579]	valid_0's auc: 0.637094
[580]	valid_0's auc: 0.63707
[581]	valid_0's auc: 0.637084
[582]	valid_0's auc: 0.637093
Early stopping, best iteration is:
[382]	valid_0's auc: 0.637336


In [165]:
y_pred_lgb = lgb_clf.predict(x_test)

In [78]:
lgb_clf.save_model('../lgb_models/lgb_1.txt')

In [166]:
gini(y_test.squeeze(), y_pred_lgb)

0.27564711328883768

### Ensembling

In [167]:
y_pred_avg = (y_pred_lgb + y_pred_catboost_trained + y_pred_xgb_trained) / 3

In [168]:
gini(y_test.squeeze(), y_pred_avg)

0.28066152662534122

In [ ]:
test_df = pd.read_csv('../data/test.csv')
id_test = test_df['id'].values

In [ ]:
# Submission File
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_pred_avg
sub.to_csv('../submissions/submission_4.csv', float_format='%.6f', index=False)